Lesson 4

---

# 🧑‍🍳 LangChain: Output Parsers

**Outline**

* Output Parser

> 🔴 <font color="red">Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the class.</font>

## ⚙️ Setup 

[OpenAI API Key](https://platform.openai.com/account/api-keys)

In [31]:
import re

# Open AI
from openai import OpenAI

# LangChain
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.chat_models import ChatOpenAI

# Local
from util import local_settings
from env_colors import TerminalTextColor

# Settings
model="gpt-3.5-turbo"

print("First LLM API example")
print(f"✅ OpenAI Key loaded (...{local_settings.OPENAI_API_KEY[20:-20]}...)")
print(f"✅ Model: {model}")


First LLM API example
✅ OpenAI Key loaded (...UQiT3BlbkFJ...)
✅ Model: gpt-3.5-turbo


> 🔔 <font color="#00d4d4">**Note:** some characters of the key are omitted for security reasons.</font>

In [32]:
client = OpenAI(api_key=local_settings.OPENAI_API_KEY)

def get_completion(prompt, temperature= 0, messages = [], model="gpt-3.5-turbo"):

    message = {"role": "user", "content": prompt}

    messages.append(message)

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

    return completion.choices[0].message.content

## Example 1: Without Output Parser

```mermaid
flowchart LR
    text --> ChatPromptTemplate
    format_instructions --> ChatPromptTemplate
    ChatPromptTemplate --> Prompt
    Prompt --> LLM"Chat"
    LLM"Chat" --> Result
    Result --> str
```
The chain view

Let's start by defining the preferred structure for the output produced by the Language Model:

In [33]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [40]:
customer_review = """
This leaf blower is pretty amazing.

It has four settings: candle blower, gentle breeze, windy city, and tornado.
It arrived in two days, just in time for my wife's anniversary present.

I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else?
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [41]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else?\nAnswer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price, and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [42]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, model=model)
response = chat(messages)

print(f"{TerminalTextColor.BLUE}----- RESPONSE ------{TerminalTextColor.RESET}")

print(response.content)

----- RESPONSE ------
{
  "gift": false,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [43]:
print(f"{TerminalTextColor.BLUE}----- TYPE ------{TerminalTextColor.RESET}")
type(response.content)

----- TYPE ------


str

<h4><u>Note:</u></h4>

Executing the following line of code will result in an error since 'gift' is a string (`str`), not a dictionary (`dict`) as required.
<h4>👇 </h4>

In [44]:
try:
    response.content.get('gift')
except:
    print(f"\
{TerminalTextColor.RED}⚠️ ERROR: \
response.content is a str not dict \
{TerminalTextColor.RESET}")

⚠️ ERROR: response.content is a str not dict 


In [45]:
import json
response_as_dict = json.loads(response.content)

print(f"{TerminalTextColor.BLUE}----- TYPE response.content------{TerminalTextColor.RESET}")
print(type(response.content))

print(f"{TerminalTextColor.BLUE}----- CONTENT response_as_dict------{TerminalTextColor.RESET}")
print(response.content)

print(f"{TerminalTextColor.BLUE}----- TYPE response_as_dict------{TerminalTextColor.RESET}")
print(type(response_as_dict))

print(f"{TerminalTextColor.BLUE}----- CONTENT response_as_dict------{TerminalTextColor.RESET}")
print(response_as_dict)


----- TYPE response.content------
<class 'str'>
----- CONTENT response_as_dict------
{
  "gift": false,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
----- TYPE response_as_dict------
<class 'dict'>
----- CONTENT response_as_dict------
{'gift': False, 'delivery_days': 2, 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}


## Example 2: WITH Output Parser

```mermaid
flowchart LR
    text --> ChatPromptTemplate
    format_instructions --> ChatPromptTemplate
    ChatPromptTemplate --> Prompt
    Prompt --> LLM"Chat"
    LLM"Chat" --> Result
    Result --> *Output_parser*
    *Output_parser* --> dict
```
The chain view

### Schemas

In [46]:
gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.")
delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days did it take for the product to arrive? If this information is not found,output -1.")
price_value_schema = ResponseSchema(
    name="price_value",
    description="Extract any sentences about the value or price, and output them as a comma separated Python list.")

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)


In [47]:

format_instructions = output_parser.get_format_instructions()
print(format_instructions)


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product to arrive? If this information is not found,output -1.
	"price_value": string  // Extract any sentences about the value or price, and output them as a comma separated Python list.
}
```


In [51]:
review_template_2 = """
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else?
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

print(review_template_2)


For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else?
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,
and output them as a comma separated Python list.

text: {text}

{format_instructions}



In [52]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)
print(prompt)

input_variables=['format_instructions', 'text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], template='\nFor the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else?\nAnswer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product\nto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,\nand output them as a comma separated Python list.\n\ntext: {text}\n\n{format_instructions}\n'))]


In [55]:

messages = prompt.format_messages(
    text=customer_review,
    format_instructions=format_instructions
)

print(messages[0].content)


For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else?
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,
and output them as a comma separated Python list.

text: 
This leaf blower is pretty amazing.

It has four settings: candle blower, gentle breeze, windy city, and tornado.
It arrived in two days, just in time for my wife's anniversary present.

I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" an

In [59]:
response = chat(messages)

print(f"{TerminalTextColor.BLUE}----- RESPONSE ------{TerminalTextColor.RESET}")
print(response.content)

----- RESPONSE ------
```json
{
	"gift": false,
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [60]:
print(f"{TerminalTextColor.BLUE}----- PARSED ------{TerminalTextColor.RESET}")

output_dict = output_parser.parse(response.content)
print(output_dict)

print(type(output_dict))

----- PARSED ------
{'gift': False, 'delivery_days': '2', 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}
<class 'dict'>


In [64]:
output_dict["gift"]

False

<h3><font color="Yellow">👋 the end </font></h3>